In [6]:
import numpy as np
from scipy.optimize import minimize
from groo.groo import get_root
import matplotlib.pyplot as plt
import pandas as pd
import sys, os
from groo.groo import get_root
rf = get_root(".hidden_root_mc")
sys.path.append(os.path.join(rf))

from models_and_funcs import *
import itertools as it 

In [192]:
# Generate data 

def gen_bandits(N=180, nbandit=3, noise=2):

    rewards = np.zeros((N,nbandit))*np.nan 
    rewards[:,0] = gen_states( lvls=[20, 20], ch=20, n=N) + np.random.normal(0, noise, N) 
    rewards[:,1] = gen_states( lvls=[20, 80], ch=20, n=N) + np.random.normal(0, noise, N) 
    rewards[:,2] = gen_states( lvls=[80, 80], ch=20, n=N) + np.random.normal(0, noise, N) 

    c = pd.DataFrame(list(it.combinations([x for x in range(nbandit)], 2)))
    reps = N//c.shape[0]
    repeated_df = pd.concat([c] * reps, ignore_index=True)
    repeated_df.columns = ["option1", "option2"]

    return rewards, repeated_df

def softmax(values, beta, parametrization="inverse"):
  denom = 0   
  if parametrization=="inverse":
    # expected rnage [0 Inf] igher values make choices more equal
    for v in values:
      denom = denom + np.exp(v/beta) 
    probs = np.exp(values/beta) / (denom)
  #http://incompleteideas.net/book/ebook/node17.html
  elif parametrization=="log_inverse":
    for v in values:
      denom = denom + np.exp(v/np.log(beta)) 
    probs = np.exp(values/np.log(beta)) / (denom)
  elif parametrization=="normal":
    for v in values:
      denom = denom + np.exp(v*beta) 
    probs = np.exp(values*beta) / (denom)
  return(probs)


def rw1_choice(params=[0.2], indata=indata):
    alpha = params[0]  # p[1] ... learning rate
    beta = params[1] # inverse temperature
    Qs = np.zeros((indata["r"].shape[0]+1,indata["r"].shape[1]))*np.nan
    Qs[0,0:3] = 50
    ntr = indata["r"].shape[0]
    choices = np.zeros((indata["r"].shape[0]))*np.nan
    ch_prob = np.zeros((indata["r"].shape[0]))*np.nan
    for tr in range(ntr):
        # get options available on this trial
        op1 = indata["options"]["option1"].iloc[tr] 
        op2 = indata["options"]["option2"].iloc[tr] 
        
        #values on this trial 
        values = Qs[tr,[op1, op2]]
        
        # choice
        probs = softmax(values, beta, parametrization="inverse")


        if any(np.isnan(probs)) | any(np.isinf(probs)):
           
           print(indata["model"])
           print(params)
           print(probs)
           print(values)
           stop= 1
        # generate choices or use probabilities to make choices
        if indata["generate_choices"] == 1:
           choice = np.random.choice([op1, op2], p=probs)
        else: 
           choice = int(indata["choices"][tr] )

        # probability of choice (for likelihood)  
        if choice == op1: 
           chosen_prob = probs[0]
        elif choice == op2: 
           chosen_prob = probs[1]
        choices[tr] = choice
        ch_prob[tr] = chosen_prob

        # update chosen
        Qs[tr+1,choice] = Qs[tr,choice] + alpha*(indata["r"][tr, choice] - Qs[tr,choice])

        #update unchosen 
        for ch in list(set(list([0,1,2])) - set(list([choice]))):
           Qs[tr+1,ch] = Qs[tr,ch]

        #Q.append(Q[o_idx] + alpha*(o - Q[o_idx])   )
    mod = {"Qs": Qs, "choices":choices, "choice_prob":ch_prob}        
    return mod

def rw2_val_choice(params=[0.2, 0.2, 1], indata=indata):
    alpha_pos = params[0]
    alpha_neg = params[1]

    beta = params[2] # inverse temperature
    Qs = np.zeros((indata["r"].shape[0]+1,indata["r"].shape[1]))*np.nan
    Qs[0,0:3] = 50
    ntr = indata["r"].shape[0]
    choices = np.zeros((indata["r"].shape[0]))*np.nan
    ch_prob = np.zeros((indata["r"].shape[0]))*np.nan
    for tr in range(ntr):
        # get options available on this trial
        op1 = indata["options"]["option1"].iloc[tr] 
        op2 = indata["options"]["option2"].iloc[tr] 
        
        #values on this trial 
        values = Qs[tr,[op1, op2]]
        
        # choice
        probs = softmax(values, beta, parametrization="inverse")

        if any(np.isnan(probs)) | any(np.isinf(probs)):
           print(indata["model"])
           print(params)
           print(probs)
           print(values)
           stop= 1
        # generate choices or use probabilities to make choices
        if indata["generate_choices"] == 1:
           choice = np.random.choice([op1, op2], p=probs)
        else: 
           choice = int(indata["choices"][tr] )

        # probability of choice (for likelihood)  
        if choice == op1: 
           chosen_prob = probs[0]
        elif choice == op2: 
           chosen_prob = probs[1]
        choices[tr] = choice
        ch_prob[tr] = chosen_prob

        # update chosen
        if (indata["r"][tr, choice] - Qs[tr,choice]) > 0:
          Qs[tr+1,choice] = Qs[tr,choice] + alpha_pos*(indata["r"][tr, choice] - Qs[tr,choice])
        elif (indata["r"][tr, choice] - Qs[tr,choice]) <= 0:
           Qs[tr+1,choice] = Qs[tr,choice] + alpha_neg*(indata["r"][tr, choice] - Qs[tr,choice])

        #update unchosen 
        for ch in list(set(list([0,1,2])) - set(list([choice]))):
           Qs[tr+1,ch] = Qs[tr,ch]

        #Q.append(Q[o_idx] + alpha*(o - Q[o_idx])   )
    mod = {"Qs": Qs, "choices":choices, "choice_prob":ch_prob}        
    return mod

def rw3_choice(params=[0.2, 0.2, 0.2, 5], indata=indata):
    # model with separate alphas for the three cues
    alpha = params[0:3]
    #alpha[0] = params[0]  # p[1] ... learning rate
    #alpha[1]= params[1]  # p[1] ... learning rate
    #alpha[2] = params[2]  # p[1] ... learning rate
    beta = params[3] # inverse temperature
    Qs = np.zeros((indata["r"].shape[0]+1,indata["r"].shape[1]))*np.nan
    Qs[0,0:3] = 50
    ntr = indata["r"].shape[0]
    choices = np.zeros((indata["r"].shape[0]))*np.nan
    ch_prob = np.zeros((indata["r"].shape[0]))*np.nan
    for tr in range(ntr):
        # get options available on this trial
        op1 = indata["options"]["option1"].iloc[tr] 
        op2 = indata["options"]["option2"].iloc[tr] 
        
        #values on this trial 
        values = Qs[tr,[op1, op2]]
        
        # choice
        probs = softmax(values, beta, parametrization="inverse")

        if any(np.isnan(probs)) | any(np.isinf(probs)):
           print(indata["model"])
           print(params)
           print(probs)
           print(values)
           stop= 1
        # generate choices or use probabilities to make choices
        if indata["generate_choices"] == 1:
           choice = np.random.choice([op1, op2], p=probs)
        else: 
           choice = int(indata["choices"][tr] )

        # probability of choice (for likelihood)  
        if choice == op1: 
           chosen_prob = probs[0]
        elif choice == op2: 
           chosen_prob = probs[1]
        choices[tr] = choice
        ch_prob[tr] = chosen_prob

        # update chosen
        Qs[tr+1,choice] = Qs[tr,choice] + alpha[choice]*(indata["r"][tr, choice] - Qs[tr,choice])

        #update unchosen 
        for ch in list(set(list([0,1,2])) - set(list([choice]))):
           Qs[tr+1,ch] = Qs[tr,ch]

        #Q.append(Q[o_idx] + alpha*(o - Q[o_idx])   )
    mod = {"Qs": Qs, "choices":choices, "choice_prob":ch_prob}        
    return mod

def rw6_val_choice(params=[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 5], indata=indata):
    # model with separate alphas for the three cues
    alpha_pos = params[0:3]
    alpha_neg = params[3:6]
    beta = params[6] # inverse temperature
    Qs = np.zeros((indata["r"].shape[0]+1,indata["r"].shape[1]))*np.nan
    Qs[0,0:3] = 50
    ntr = indata["r"].shape[0]
    choices = np.zeros((indata["r"].shape[0]))*np.nan
    ch_prob = np.zeros((indata["r"].shape[0]))*np.nan
    for tr in range(ntr):
        # get options available on this trial
        op1 = indata["options"]["option1"].iloc[tr] 
        op2 = indata["options"]["option2"].iloc[tr] 
        
        #values on this trial 
        values = Qs[tr,[op1, op2]]
        
        # choice
        probs = softmax(values, beta, parametrization="inverse")

        if any(np.isnan(probs)) | any(np.isinf(probs)):
           print(indata["model"])
           print(params)
           print(probs)
           print(values)
           stop= 1
           
        # generate choices or use probabilities to make choices
        if indata["generate_choices"] == 1:
           choice = np.random.choice([op1, op2], p=probs)
        else: 
           choice = int(indata["choices"][tr] )

        # probability of choice (for likelihood)  
        if choice == op1: 
           chosen_prob = probs[0]
        elif choice == op2: 
           chosen_prob = probs[1]
        choices[tr] = choice
        ch_prob[tr] = chosen_prob

        # update chosen
        if (indata["r"][tr, choice] - Qs[tr,choice]) > 0:
          Qs[tr+1,choice] = Qs[tr,choice] + alpha_pos[choice]*(indata["r"][tr, choice] - Qs[tr,choice])
        elif (indata["r"][tr, choice] - Qs[tr,choice]) <= 0:
           Qs[tr+1,choice] = Qs[tr,choice] + alpha_neg[choice]*(indata["r"][tr, choice] - Qs[tr,choice])

        #update unchosen 
        for ch in list(set(list([0,1,2])) - set(list([choice]))):
           Qs[tr+1,ch] = Qs[tr,ch]

        #Q.append(Q[o_idx] + alpha*(o - Q[o_idx])   )
    mod = {"Qs": Qs, "choices":choices, "choice_prob":ch_prob}        
    return mod



def lklhd_choice(params, indata):
    model = indata["model"]
    m1 = model(params, indata)
    ll=np.sum(-np.log(m1["choice_prob"]))
    return ll


def lklhd_choice_m(params, indata):
    # 1/ evaluate model given participant's choices 
    model = indata["model"]
    indata["data_choices"] = indata["choices"]
    m1 = model(params, indata)
    negLL=np.sum(-np.log(m1["choice_prob"]))
   
    n = indata["r"].shape[0]
    m1["n"] = n 
    
    m1["negLL"] = negLL
    noparams = len(params)
    m1["noparams"] = noparams
    m1["AIC"] = 2*noparams + 2*negLL #ll is already negative log, thus +
    
    
    m1["AICc"] = m1["AIC"] + ((2*noparams**2 + 2*noparams ) / (n - noparams - 1))
    m1["BIC"] = noparams*np.log(n) + 2*negLL #ll is already negative log, thus +
    m1["HQC"] = 2*negLL + 2*noparams*np.log(np.log(n))

    # 2/ use parameters to get choice accuracy measure
    indata2=indata 
    indata2["generate_choices"]=1
    m2 = model(params, indata2)
    m1["acc"] = np.mean(m2["choices"] == indata["data_choices"])
    return m1

def replace_random_values(arr, proportion):
    num_to_replace = int(len(arr) * proportion)
    indices_to_replace = np.random.choice(len(arr), num_to_replace, replace=False)
    arr[indices_to_replace] = np.random.randint(3, size=num_to_replace)
    return arr

# Make choices
N = 180
# Generate data 
rewards, repeated_df = gen_bandits(N, nbandit=3)
indata = {"options": repeated_df, "r":rewards, "nbandit":3, "model":rw6_val_choice, "generate_choices":1} 
# make choices
m = rw6_val_choice([0.2,0.2, 0.2, 0.8,0.8, 0.8, 2], indata)
indata["choices"] = m["choices"]
indata["generate_choices"] = 0
print(lklhd_choice([0.2,0.2, 0.2, 0.8,0.7, 0.8, 2], indata))
print(lklhd_choice([0.2,0.2, 0.2, 0.8,0.8, 0.8, 2], indata))

m1=lklhd_choice_m([0.2,0.2, 0.2, 0.8,0.6, 0.8, 2], indata)
print(m1["acc"])
m2=lklhd_choice_m([0.2,0.2, 0.2, 0.8,0.8, 0.8, 1], indata)
print(m2["acc"])

20.200392413848235
19.12654234068733
0.9277777777777778
0.9166666666666666


In [98]:
bnds = ((0,1),(0.0001,50))
#bnds = ((0,100),(0,1),(0,1))
opt = minimize(fun=lklhd_choice, x0=[0.2, 5], args=(indata), method='Nelder-Mead', bounds=bnds, options={'verbose': 0})
print(opt.message)
print(opt.x)

/tmp/ipykernel_370341/3250818362.py:3: OptimizeWarning: Unknown solver options: verbose
  opt = minimize(fun=lklhd_choice, x0=[0.2, 5], args=(indata), method='Nelder-Mead', bounds=bnds, options={'verbose': 0})


Maximum number of function evaluations has been exceeded.
[0.19608551 5.12440556]


In [171]:
rewards[178:,:]

array([[18.57149425, 17.61402364, 82.02045283],
       [20.57788344, 20.30166931, 83.23943877]])

In [181]:
vals = np.array([0.3, np.nan])
if any(np.isnan(vals)) | any(np.isinf(vals)):
    stop= 1
np.isin

True

### test model runs

In [193]:

cond = "test"
alg = "COBYLA"
noise = 0.2
c = 60
iterations = 5
cond_str = "mc_n"+str(noise)+"_c"+str(c)+"_i"+str(iterations)+"_"+alg

import numpy as np
from scipy.optimize import minimize
from groo.groo import get_root
import matplotlib.pyplot as plt
import pandas as pd
import sys, os
from groo.groo import get_root
rf = get_root(".hidden_root_mc")
sys.path.append(os.path.join(rf))

from models_and_funcs import *

N = 180
#noise_level= [5, 10, 20]
#cutoff = [90, 120, 150]
models = [rw1_choice, rw2_val_choice, rw3_choice, rw6_val_choice] 
model_names = ["rw1_choice", "rw2_val_choice", "rw3_choice", "rw6_val_choice"]
bounds = {"rw1_choice": ((0,1),(0.001,50)),
          "rw2_val_choice": ((0,1),(0,1),(0.001,50)), 
          "rw3_choice": ((0,1),(0,1),(0,1),(0.001,50)), 
          "rw6_val_choice": ((0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0.001,50))}
df = pd.DataFrame()


for ii in range(iterations):
    print(np.round(ii/iterations,2))
    # Generate data
    rew_all, options_all = gen_bandits(N, nbandit=3)
    #indata = {"options": repeated_df, "r":rewards, "nbandit":3, "model":rw6_val_choice, "generate_choices":1} 

    # trainign data 
    rw_train = rew_all[0:c,:]
    op_train = options_all.iloc[0:c, :]

    # testing data 
    rw_test = rew_all[c:,:]
    op_test = options_all.iloc[c:, :]

    for m_idx, (m, mname) in enumerate(zip(models, model_names)): # loop over model GENERATING the data
        params_in = gen_rand_vals(bounds[mname])
        indata = {"options": options_all, "r":rew_all, "nbandit":3, "model":m, "generate_choices":1} 

        mpred = m(params_in, indata)

        #induce noice by randomly replaceing proportion of choices
        mpred["choices"] = replace_random_values(mpred["choices"], noise)

        ch_train = mpred["choices"][0:c]
        ch_test = mpred["choices"][c:]

        # Fit data 
        AIC =[] 
        BIC = []
        AICc = [] 
        HQC = [] #https://en.wikipedia.org/wiki/Hannan%E2%80%93Quinn_information_criterion
        P = {}
        for mfit, mname_fit in zip(models, model_names):
            # prepare data generated above for fit - all trainingn    
            indata = {"options": op_train, "r":rw_train, "nbandit":3, "model":mfit, "generate_choices":0, "choices":ch_train} 

            # fit
            opt = minimize(fun=lklhd_choice, x0=gen_rand_vals(bounds[mname_fit]), args=(indata), method=alg, bounds=bounds[mname_fit], options={'verbose': 0})

            # Get IC 
            M = lklhd_choice_m(opt.x, indata)

            AIC.append(M["AIC"])
            P[mname_fit] = opt.x
            BIC.append(M["BIC"])
            AICc.append(M["AICc"])
            HQC.append(M["HQC"])
           
        
        # which model fit best
        best_idx =[np.argmin(AIC), np.argmin(AICc), np.argmin(BIC), np.argmin(HQC)]
        # is this the correct model
        #if aic_idx == m_idx:
        #    m_recovery_AIC = 1 # etc

        # get predictive error per trial of the best model
        acc = []
        for idxx, best in enumerate(best_idx):
            #test all 
            indata = {"options": op_test, "r":rw_test, "nbandit":3, "model":models[best_idx[idxx]], "generate_choices":0, "choices":ch_test} 
            Mbest = lklhd_choice_m(P[model_names[best_idx[idxx]]], indata)
            acc.append(Mbest["acc"])

        # Gather data
        D = {"noise": noise, "cutoff":c, "true_model": mname, "algo":alg,
            "best_model_AIC": model_names[best_idx[0]], 
            "best_model_AICc": model_names[best_idx[1]],  
            "best_model_BIC": model_names[best_idx[2]], 
            "best_model_HQC": model_names[best_idx[3]], 
            "acc_AIC": acc[0],
            "acc_AICc": acc[1],
            "acc_BIC": acc[2], 
            "acc_HQC": acc[3]                  
            }
        dfrow = pd.DataFrame.from_dict(D, orient="index").T
        df = pd.concat([df, dfrow], axis=0)
df.to_csv(os.path.join(rf, "data", cond, "model_comparison_iter"+cond_str+".csv") )

0.0


/home/ondrej/anaconda3/envs/python3.10/lib/python3.10/site-packages/scipy/optimize/_minimize.py:572: RuntimeWarning: Method COBYLA cannot handle bounds.
  warn('Method %s cannot handle bounds.' % method,
/home/ondrej/anaconda3/envs/python3.10/lib/python3.10/site-packages/scipy/optimize/_minimize.py:705: OptimizeWarning: Unknown solver options: verbose
  res = _minimize_cobyla(fun, x0, args, constraints, callback=callback,
/home/ondrej/anaconda3/envs/python3.10/lib/python3.10/site-packages/scipy/optimize/_minimize.py:572: RuntimeWarning: Method COBYLA cannot handle bounds.
  warn('Method %s cannot handle bounds.' % method,
/home/ondrej/anaconda3/envs/python3.10/lib/python3.10/site-packages/scipy/optimize/_minimize.py:705: OptimizeWarning: Unknown solver options: verbose
  res = _minimize_cobyla(fun, x0, args, constraints, callback=callback,
/tmp/ipykernel_370341/1144863027.py:22: RuntimeWarning: overflow encountered in exp
  denom = denom + np.exp(v/beta)
/tmp/ipykernel_370341/114486302

<function rw1_choice at 0x7f3b5d37e830>
[-0.41779388 14.24705621]
[nan  0.]
[  14229.95305508 -373509.48616323]
<function rw1_choice at 0x7f3b5d37e830>
[-0.41779388 14.24705621]
[ 0. nan]
[  540.53828726 20166.39130743]
<function rw1_choice at 0x7f3b5d37e830>
[-0.41779388 14.24705621]
[nan  0.]
[  20166.39130743 -529592.68763094]
<function rw1_choice at 0x7f3b5d37e830>
[-0.41779388 14.24705621]
[ 0. nan]
[  758.28691195 28583.78392373]
<function rw1_choice at 0x7f3b5d37e830>
[-0.41779388 14.24705621]
[nan  0.]
[  57435.81381529 -529592.68763094]
<function rw6_val_choice at 0x7f3b5d37de10>
[ 5.48530938e-01  1.81010789e-02  5.74663243e-01  5.03232635e-01
 -8.09503226e-01  1.46313077e+00  3.61726437e+01]
[nan  0.]
[35767.02094907    80.71212382]
<function rw6_val_choice at 0x7f3b5d37de10>
[ 5.48530938e-01  1.81010789e-02  5.74663243e-01  5.03232635e-01
 -8.09503226e-01  1.46313077e+00  3.61726437e+01]
[ 0. nan]
[1.95155110e+01 3.57670209e+04]
<function rw6_val_choice at 0x7f3b5d37de10>
[ 

/home/ondrej/anaconda3/envs/python3.10/lib/python3.10/site-packages/scipy/optimize/_minimize.py:572: RuntimeWarning: Method COBYLA cannot handle bounds.
  warn('Method %s cannot handle bounds.' % method,
/home/ondrej/anaconda3/envs/python3.10/lib/python3.10/site-packages/scipy/optimize/_minimize.py:705: OptimizeWarning: Unknown solver options: verbose
  res = _minimize_cobyla(fun, x0, args, constraints, callback=callback,
/tmp/ipykernel_370341/1144863027.py:22: RuntimeWarning: overflow encountered in exp
  denom = denom + np.exp(v/beta)
/tmp/ipykernel_370341/1144863027.py:23: RuntimeWarning: overflow encountered in exp
  probs = np.exp(values/beta) / (denom)
/tmp/ipykernel_370341/1144863027.py:23: RuntimeWarning: invalid value encountered in divide
  probs = np.exp(values/beta) / (denom)
/tmp/ipykernel_370341/1144863027.py:258: RuntimeWarning: divide by zero encountered in log
  ll=np.sum(-np.log(m1["choice_prob"]))


<function rw6_val_choice at 0x7f3b5d37de10>
[ 0.52032659  0.66653903 -0.02204866 -0.66052272  0.14307953  1.84785247
 12.737408  ]
[nan  0.]
[13818.68400703    52.64908541]
<function rw6_val_choice at 0x7f3b5d37de10>
[ 0.52032659  0.66653903 -0.02204866 -0.66052272  0.14307953  1.84785247
 12.737408  ]
[nan  0.]
[13818.68400703    32.18632707]
<function rw6_val_choice at 0x7f3b5d37de10>
[ 0.52032659  0.66653903 -0.02204866 -0.66052272  0.14307953  1.84785247
 12.737408  ]
[nan  0.]
[22932.86258372    44.17741825]
<function rw6_val_choice at 0x7f3b5d37de10>
[ 0.52032659  0.66653903 -0.02204866 -0.66052272  0.14307953  1.84785247
 12.737408  ]
[nan  0.]
[3.80682914e+04 3.21863271e+01]
<function rw6_val_choice at 0x7f3b5d37de10>
[ 0.52032659  0.66653903 -0.02204866 -0.66052272  0.14307953  1.84785247
 12.737408  ]
[nan  0.]
[6.31989094e+04 4.41774183e+01]
<function rw6_val_choice at 0x7f3b5d37de10>
[ 0.52032659  0.66653903 -0.02204866 -0.66052272  0.14307953  1.84785247
 12.737408  ]
[nan

/home/ondrej/anaconda3/envs/python3.10/lib/python3.10/site-packages/scipy/optimize/_minimize.py:572: RuntimeWarning: Method COBYLA cannot handle bounds.
  warn('Method %s cannot handle bounds.' % method,
/home/ondrej/anaconda3/envs/python3.10/lib/python3.10/site-packages/scipy/optimize/_minimize.py:705: OptimizeWarning: Unknown solver options: verbose
  res = _minimize_cobyla(fun, x0, args, constraints, callback=callback,
/tmp/ipykernel_370341/1144863027.py:22: RuntimeWarning: overflow encountered in exp
  denom = denom + np.exp(v/beta)
/tmp/ipykernel_370341/1144863027.py:23: RuntimeWarning: overflow encountered in exp
  probs = np.exp(values/beta) / (denom)
/tmp/ipykernel_370341/1144863027.py:23: RuntimeWarning: invalid value encountered in divide
  probs = np.exp(values/beta) / (denom)
/tmp/ipykernel_370341/1144863027.py:258: RuntimeWarning: divide by zero encountered in log
  ll=np.sum(-np.log(m1["choice_prob"]))


0.2


/home/ondrej/anaconda3/envs/python3.10/lib/python3.10/site-packages/scipy/optimize/_minimize.py:572: RuntimeWarning: Method COBYLA cannot handle bounds.
  warn('Method %s cannot handle bounds.' % method,
/home/ondrej/anaconda3/envs/python3.10/lib/python3.10/site-packages/scipy/optimize/_minimize.py:705: OptimizeWarning: Unknown solver options: verbose
  res = _minimize_cobyla(fun, x0, args, constraints, callback=callback,
capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.


KeyboardInterrupt: 